# A Convolutional Neural Network

In [1]:
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import (
    Input, Dense, Dropout, Activation, Flatten,
    Conv2D, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom, RandomTranslation
)
from keras.optimizers import RMSprop

Define and initialize variables

In [2]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

# constants
BATCH_SIZE = 128
NB_EPOCH = 60
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()
NUM_TO_AUGMENT = 5

Load the dataset, do one-hot encoding, and normalize the images

In [3]:
# load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert to categorical
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)

# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Add augmentation as part of the model

In [4]:
data_augmentation = Sequential([
    RandomFlip('horizontal'),
    RandomRotation(0.05),
    RandomZoom(0.1),
    RandomTranslation(0.1, 0.1)
])

CNN config

In [5]:
# network
model = Sequential()
model.add(Input((IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(data_augmentation)
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,606,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,676,842 (6.40 MB)

 Trainable params: 1,676,842 (6.40 MB)

 Non-trainable params: 0 (0.00 B)

Train the model

In [6]:
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(
    X_train, 
    Y_train, 
    batch_size=BATCH_SIZE,
    epochs=NB_EPOCH,
    validation_split=VALIDATION_SPLIT,
    verbose=VERBOSE
)
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 61ms/step - accuracy: 0.2236 - loss: 2.1070 - val_accuracy: 0.3775 - val_loss: 1.7816
Epoch 2/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 61ms/step - accuracy: 0.3949 - loss: 1.6596 - val_accuracy: 0.4560 - val_loss: 1.5731
Epoch 3/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 62ms/step - accuracy: 0.4739 - loss: 1.4764 - val_accuracy: 0.5380 - val_loss: 1.2853
Epoch 4/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.5116 - loss: 1.3664 - val_accuracy: 0.5796 - val_loss: 1.1765
Epoch 5/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 62ms/step - accuracy: 0.5437 - loss: 1.2787 - val_accuracy: 0.6000 - val_loss: 1.1329
Epoch 6/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.5689 - loss: 1.2100 - val_accuracy: 0.6136 - val_loss: 1.0904
Epoch 7/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.5865 - loss: 1.1620 - val_accuracy: 0.6195 - val_loss: 1.1061
Epoch 8/60
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.6058 - loss: 1.1137 - 

Save the architecture of the deep network

In [8]:
# save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
# save weights
model.save_weights('cifar10.weights.h5', overwrite=True)

#### Overview of Ethical Implications

Convolutional neural networks are a powerful way to build programs that recognize features within images. In the code above, an algorithm was created that is able to differentiate between animals and vehicles with about 75% accuracy. Admittedly, this is a mere toy example that has no real-world application. But what if the images used to train the model were those of people's faces? What would be the ethical concerns and implications?

In 2017, Clearview AI launched an application capable of identifying individuals within seconds (Blum, 2024). The AI model that powers the software was trained on billions of publicly available images. The tool was supposedly only going to be used by retailers to identify shoplifters but was eventually licensed to hundreds of law enforcement agencies across the United States (Blum, 2024). Understandably, this set off a chain reaction of media coverage and several lawsuits. How could the company guarantee that the software would not be exploited by potential bad actors? Who was vetting Clearview AI to ensure that the AI practices being used were ethical?

Some of the major concerns regarding facial recognition technologies (FRT) involve the collection of data used for these systems and the transparency of their usage. Several members of University College London published a comprehensive overview of the ethical considerations as well as a collection of poignant questions for those developing and using such systems. What constitutes fair use of data scraped, how accountability will be implemented, and what sort of auditing will be enabled were some of the critical questions posed that should be considered by any deployer or user of FRT (Almeida et al., 2023). Although FRT has exciting and powerful applications, great care must be taken to ensure that no ethical boundaries are crossed and that everyone's right to privacy is respected.

**Resources**

Almeida, D., Shmarko, K., & Lomas, E. (2022). The ethics of facial recognition technologies, surveillance, and accountability in an age of artificial intelligence: A comparative analysis of US, EU, and UK regulatory frameworks. *AI and Ethics*, 2(3), 377–387. https://doi.org/10.1007/s43681-021-00077-w

Blum, S. (2024, August 12). *Clearview AI’s founder on the company’s controversial beginnings and massive growth.* Inc. https://www.inc.com/sam-blum/clearview-ais-founder-on-companys-controversial-beginnings-massive-growth-since.html